In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict ['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = ["section","text","question"],
    keyword_fields = ['course']
)

In [7]:
q = 'the course has already started. can i still enrol?'

In [8]:
index.fit(documents)

In [9]:
import os
from huggingface_hub import InferenceClient

In [10]:
client = InferenceClient()

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict= {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [23]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ question. 
Use only the facts from the CONTEXT when answering the question.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question =query, context=context).strip()
    return prompt

In [24]:
def llm(prompt):
    response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3-0324",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

    return response.choices[0].message.content

In [27]:
query = 'the course just started, how do i enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag (query)

'To enroll in the course before it starts, use the registration link provided: **"Register before the course starts using this link."**  \n\nAdditionally, ensure you:  \n whether third of all \n whether third of all  \n \n\n1. Subscribe to the course\'s Google Calendar for updates.  \n2. Join the course Telegram channel for announcements.  \n3. Register in DataTalks.Club\'s Slack and join the channel.  \n\nThese steps are from the FAQ section under *General course-related questions*.'

In [29]:
rag('how do i register for the course')

"To register for the course, use the registration link provided in the FAQ. The course starts on 15th Jan 2024 at 17h00, and you should register before it begins. Additionally, subscribe to the course's public Google Calendar, join the Telegram channel for announcements, and register in DataTalks.Club's Slack to join the channel. \n\nFor more details, refer to the FAQ section on course registration and pre-course actions."

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [63]:
from elasticsearch import Elasticsearch

In [64]:
es_client = Elasticsearch(
    "http://localhost:9200",
)

In [65]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question"

es_client.indices.create(index=index_name, body= index_settings)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)

In [66]:
import elasticsearch
print(elasticsearch.__version__)


(9, 1, 0)
